In [4]:
import pandas as pd
import ftfy
from pathlib import Path

In [16]:
folders = []
input_folder = Path('fb-messages')
msg_files = []
for x in input_folder.iterdir():
    folders.append(Path(x))
    
for y in folders:
    z = list(y.iterdir())
    for a in z:
        if a.rglob("*.json"):
            msg_files.append(Path(a))

In [20]:
names_df = []

In [21]:
def get_second(x):
    return x[1].rstrip('.')

In [22]:
def get_nick_names(file_path):
    df = pd.read_json(file_path, typ="series")
    df = pd.Series(df["messages"])
    df = pd.json_normalize(df)
    df = df[["content", "sender_name", "timestamp_ms"]]
    df = df.fillna("")
    df["content"] = df.content.apply(ftfy.fix_text) 
    df = df[df["content"].str.contains("nickname to") == True]
    df["content"] = df.content.str.split("nickname to")
    df["content"] = df["content"].apply(get_second).str.strip(" ")
    names_df.append(df)


In [23]:
for x in msg_files: get_nick_names(x)

In [24]:
df = pd.concat(names_df, ignore_index=True)

In [26]:
df.to_json("./output/nicknames.json", orient="records", force_ascii=False)